In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/mnist-classification-master'

/content/drive/MyDrive/mnist-classification-master


In [ ]:
ls

flask-app/  mnist_cnn.h5  mnist_cnn.py  none.ipynb  README.md


In [ ]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Activation, Flatten, Dropout, MaxPool2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from keras.utils.vis_utils import model_to_dot 
from keras.utils.vis_utils import plot_model
#from keras.utils import plot_model
from IPython.display import SVG
from keras.models import load_model
from sklearn.metrics import confusion_matrix, f1_score, precision_score, roc_auc_score
import os

np.random.seed(100)


def show_history(acc, loss):
    plt.plot(acc)
    plt.plot(loss)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'test_accuracy'], loc='best')
    plt.show()


(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

m = x_train[0]


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalize
x_train /= 255
x_test /= 255


y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

if os.path.exists('mnist_cnn.h5'):
    print('Model existed! Load model from file')
    model = load_model('mnist_cnn.h5')
else:
    print('Train new model')

    model = Sequential()


    model.add(Conv2D(filters=32,kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64,kernel_size=(3,3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(rate=0.25))
    model.add(Flatten())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=10, activation='softmax'))

    plot_model(model, to_file='model.png',show_shapes=True)

    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True), metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_train, y_train))
    model.save('mnist_cnn.h5')

    acc_data = np.array(history.history['acc'])
    loss_data = np.array(history.history['acc'])

    np.save(open('data1.txt', 'w'), acc_data)
    np.save(open('data2.txt', 'w'), loss_data)

#x = np.load(open('data1.txt'))
#y = np.load(open('data2.txt'))
#show_history(x,y)


print('Evaluating model')

score = model.evaluate(x_test, y_test, verbose=1)
prediction = model.predict(x_test)


prediction = np.argmax(prediction, axis=1)
y_test = np.argmax(y_test,axis=1)

matrix = confusion_matrix(y_test, prediction)
f1score = f1_score(y_test, prediction, average='weighted')
precision = precision_score(y_test, prediction,average='weighted')
#auc_value = roc_auc_score(y_test, precision)


print('Test score {0}'.format(score))
print('F1 score {0}'.format(f1score))
print('Precision score {0}'.format(precision))
#print('AUC score {0}'.format(auc_value))
print('Confusion matrix:\n{0}'.format(matrix))

11501568/11490434 [==============================] - 0s 0us/step
Model existed! Load model from file
Evaluating model
313/313 [==============================] - 15s 8ms/step - loss: 0.1873 - accuracy: 0.9481
Test score [0.18725836277008057, 0.9480999708175659]
F1 score 0.948031274901191
Precision score 0.948338780499506
Confusion matrix:
[[ 967    0    1    0    0    3    1    1    7    0]
 [   0 1115    3    2    1    0    4    0    9    1]
 [   9    4  953   11    7    2    5   22   19    0]
 [   0    1   12  964    0    7    0    9   15    2]
 [   1    1    4    0  937    0   11    3    3   22]
 [   3    1    1   32    2  830    7    1   13    2]
 [  12    3    2    0    4   16  919    0    2    0]
 [   2    6   30    6    2    1    0  962    6   13]
 [   4    0    4   12    4    4    3    7  928    8]
 [  10    7    6   11   24    5    0   19   21  906]]


In [ ]:
cd '/content/drive/MyDrive/mnist-classification-master/flask-app'

/content/drive/MyDrive/mnist-classification-master/flask-app


In [ ]:
ls

app.py  app.pyc  sample.png  static/  templates/  uploads/


In [ ]:
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request
from flask import render_template, jsonify
import tensorflow as tf
from tensorflow import keras
import base64
import json
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras import backend
#from keras.models import load_model
#import tensorflow as tf
import os


dir_path = '/content/drive/MyDrive/mnist-classification-master/flask-app'
UPLOAD_FOLDER = dir_path + '/uploads'
STATIC_FOLDER = dir_path + '/static'

graph = tf.compat.v1.get_default_graph()
with graph.as_default():
   model = load_model(STATIC_FOLDER + '/' + 'mnist_cnn.h5')

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def index():
    return render_template('index.html')


@app.route('/upload', methods=['GET', 'POST'])
def upload():

    if request.method == 'GET':
        return render_template('index.html')
    else:

        data = request.form['image']
        data = data.replace('data:image/png;base64,','')
        data = data.replace(' ', '+')
        imgdata = base64.b64decode(data)
        filename = UPLOAD_FOLDER + '/test.jpg'
        with open(filename, 'wb') as f:
            f.write(imgdata)

        result = predict(filename)
        predicted_number = np.asscalar(np.argmax(result, axis=1))
        accuracy = round(result[0][predicted_number] * 100, 2)

        return jsonify({'label':predicted_number},)


@app.route('/predict', methods=['GET', 'POST'])
def predict(filename):
    img = Image.open(filename)
    img = img.resize((28, 28), Image.ANTIALIAS)
    img = np.asarray(img)
    data = img[:, :, 3] # select only visible channel

    data = np.expand_dims(data, axis=2)
    data = np.expand_dims(data, axis=0)

    with graph.as_default():
        predicted = model.predict(data)

    return predicted


if __name__ == '__main__':
    #app.debug = True
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f85accb6499e.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [02/Jul/2021 09:41:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jul/2021 09:41:51] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Jul/2021 09:41:52] "GET / HTTP/1.1" 200 -
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
[2021-07-02 09:41:57,498] ERROR in app: Exception on /upload [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(